# Quickstart

These quick instructions will get you up and running with Datapane in a few minutes! 

In this quickstart, we will introduce you to the concept of Blocks, create a simple report, turn it into a data app that let's a user create a scatter plot of selected features from the popular [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set).

## Installation

First, install Datapane using pip. Check out our [installation page](/install-datapane/) for installation options. 


!!! note
    If you don't have a Python environment, you can get started for free on our hosted [Codespaces environment](https://try.datapane.com). You can also download this Quickstart as a [Jupyter Notebook](https://raw.githubusercontent.com/datapane/datapane/main/public-docs/docs/quickstart.ipynb){:download=quickstart.ipynb}.


```python
pip3 install -U datapane
```

## Setting things up

In [ ]:
import altair as alt
import datapane as dp
from vega_datasets import data

We've imported `datapane`, the popular visualization library `altair`, and `vega_datasets` which contains some sample datasets.

Let's load the Iris dataset and get a list of the features.

In [ ]:
df = data.iris()
columns = list(df.columns)
print(columns)

## Blocks

Datapane is built around the concept of _Blocks_, which are Python objects that represent an individual unit that can be processed, composed, and viewed. There are display blocks, such as _Plot_ or _DataTable_, and layout blocks, such as _Select_ and _Group_.

Having loaded our DataFrame above and with knowledge of our column names, we first create a simple scatterplot using the Altair plotting library. 

We then build a simple set of blocks which presents two tabs: one with our plot, and one with our DataFrame.

In [ ]:
fig = (
    alt.Chart(df)
    .mark_point()
    .encode(x=alt.X("sepalLength", scale=alt.Scale(zero=False)), 
            y=alt.X("sepalWidth", scale=alt.Scale(zero=False)),
            color="species")
)

view = dp.Select(dp.Plot(fig, label="Plot"), dp.DataTable(df, label="Data"))
view

## Reports

Once we have a view, we can save it as an HTML report or upload it to Datapane Cloud to share.

Let's save it as a report and [open it in a new window](./quickstart_report.html){:target=_blank}.

In [ ]:
dp.save_report(view, "quickstart_report.html", open=True)

## Apps

The blocks above that make up our report are static. This has the benefit of not requiring a running server, but the limitation of not allowing backend processing. 

To turn your report into an app, Datapane allows you to attach Python functions to your blocks and pass data into them to dynamically process and return new blocks. Functions can be run in response to forms or front-end events (such as on a schedules, or on a pageload).

### Functions

Let's build a simple function which takes two columns of the dataset as parameters, and returns a scatter plot based on these inputs.

In [ ]:
def plot_df(x_axis: str, y_axis: str, color: str) -> dp.Plot:
    # global dataset
    fig = (
        alt.Chart(df)
        .mark_point()
        .encode(
            x=alt.X(x_axis, scale=alt.Scale(zero=False)),
            y=alt.X(y_axis, scale=alt.Scale(zero=False)),
            color=color,
            tooltip=columns,
        )
    )

    return dp.Plot(fig, name="plot")

plot_df(x_axis="sepalLength", y_axis="sepalWidth", color="species")

To highlight the minor differences between a _regular_ Python function, and a function that is ready for Datapane interactivity:

- The parameters to the function should be the same as the `Controls` that we'll add to our form. Alternatively, your function can take a single `params` dictionary which contains all parameters.
- Functions must return an object which can be wrapped up as a Datapane block. This can either be a single block (such as `dp.Plot`), a list of blocks, or an object which Datapane can automatically convert to a block (such as a pandas DataFrame)


!!! note 
    Datapane blocks have full notebook support, meaning they can be displayed in notebooks as seen above. This makes it easy to interactively develop your function, and call it from your notebook with test parameters before wiring it into your Datapane App.

### Controls

Let's add some controls that let the user select which features to plot. Our feature list from earlier, `columns`, will be useful here with the the `Choice` parameter that allows the user the select an item from the list.

In [ ]:
controls = dp.Controls(
    x_axis=dp.Choice(options=columns),
    y_axis=dp.Choice(options=columns),
    color=dp.Choice(options=columns),
)

As we can see, the name of each `dp.Choice` is used as the parameter name when calling our function.

### Bringing it Together

We have our function, and our controls, now let's bring them together to create a data app with a `Form` Block.

In [ ]:
view = dp.View(
    dp.Text("# Iris Dataset Plotter"),
    dp.Form(plot_df, controls=controls),
)

Let's walk through the code. As in the report example, we are creating a list of _Blocks_ but this time we have a new Block called `Form`, which is a Compute Block that allows us to call a function from our view. 

It takes the function we created earlier, `plot_df`, and the controls we created, `controls`. When the user submits the form, the function is run with the parameters and, by default, the blocks returned as the output of our function are rendered below the Form in the UI.

We can preview the design of our data app in this notebook (but the functions won't work until we publish it.)

In [ ]:
view

## Launching the App

There are many ways to publish a Datapane app, but for now, we can use the `dp.serve_app` function to see it in action locally.

```python
dp.serve_app(view)
```

That's it! You've learnt about Blocks, Views, Reports, Apps, and even built and served your first data app with Datapane.

## Next Steps
- [View the Gallery](https://datapane.com/gallery)
- [Sign up for a free account](https://cloud.datapane.com/accounts/signup)
- [Read the documentation](https://docs.datapane.com)
- [Read our forums](https://forum.datapane.com/)